<center>
<h1> Sistema de Recomendación de Spotify 
<h2> Grupo 1 - Inteligencia Artificial 
<h2> Proyecto Final </h2>
   </center>

*Integrantes:*
- 1
- 2
- 3

>Este proyecto fue programado con la versión de <u>_Python 3.10.0

## Recolección de los datos
A través de la API de Spotify, en principio la API nos muestra información asociada a la cuenta personal, por ejemplo, artistas, albums y pistas. Para esto hay que acceder a tu cuenta de *Spotify* desde la página de desarrollador <u>*developer.spotify.com*</u>, ya está dentro de la app es necesario hacer procesos que omitiremos en este documento.

Para empezar, es necesario instalar ***spotipy*** para cargar las librerías necesariar para hacer la conexión con la API de Spotify:
* ```pip install spotipy ``` 
* ```- python3 -m pip install spotipy``` (en caso de que no te funcione la primera)


In [47]:
!pip install spotipy

Defaulting to user installation because normal site-packages is not writeable


In [48]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

A continuación guardaremos las claves que nos provee Spotify para conectarnos por medio de la API.

In [49]:
cid = "70c4eff7ab204aeaaa1ee54e6304f766" # Client ID
secret = "55944900bfbc479283e1e4f88223d854" # Client Secret ID
username = "Sebastian Gil Sanchez" # Nombre de usuario 
redirect_uri = "http://localhost:8080" # URI

In [50]:
# PRUEBA DE OTRO USUARIO
#username = "Angelo Aguilera Avila"
#username

Para poder conectarnos con Spotify vamos a crear una clase que se conecta con la clase que Spotify ya tiene, para esto necesitarmos definir unos privilegios que son necesarios para acceder a cierta información de la cuenta. 

In [112]:
scope = 'user-top-read' # Permiso que mira Spotify para dejarnos usar la funciones
token = util.prompt_for_user_token(username,scope,cid,secret,redirect_uri) # Token para crear el objeto que hace la conexión
sp = spotipy.Spotify(auth=token) # Objeto que nos permitira acceder a la info de Spotify

## ¡¡Ya tenemos acceso a la API!!
Ahora vamos a ver que tipo de información nos da Spotify:
* ``` sp.current_user_top_tracks```
* ``` sp.user_current_followed_artists```
* ``` sp.current_user_playlist```
* ``` sp.current_user_recently_played```
* ``` sp.current_user_top_artist```
* otros...
    
Para empezar vamos a usar el top de las canciones escuchadas por el usuario principal
> Al ser ```sp``` una clase Spotify, podemos ver sus objetos con el comando ```dir(sp)```

In [52]:
top20 = sp.current_user_top_tracks(time_range='short_term', limit = 20)

De acá nos podemos dar de cuenta de todas las variables que tenemos disponibles del top 20.

In [132]:
top20 # Diccionario

{'items': [{'album': {'album_type': 'SINGLE',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2uSVRVjYZ0PlJPdFHnBETu'},
      'href': 'https://api.spotify.com/v1/artists/2uSVRVjYZ0PlJPdFHnBETu',
      'id': '2uSVRVjYZ0PlJPdFHnBETu',
      'name': 'creamy',
      'type': 'artist',
      'uri': 'spotify:artist:2uSVRVjYZ0PlJPdFHnBETu'},
     {'external_urls': {'spotify': 'https://open.spotify.com/artist/31mtcsgS3CJW5dEPlG83E7'},
      'href': 'https://api.spotify.com/v1/artists/31mtcsgS3CJW5dEPlG83E7',
      'id': '31mtcsgS3CJW5dEPlG83E7',
      'name': 'untrusted',
      'type': 'artist',
      'uri': 'spotify:artist:31mtcsgS3CJW5dEPlG83E7'},
     {'external_urls': {'spotify': 'https://open.spotify.com/artist/2MDj296KJIfgWDNBtHzeFi'},
      'href': 'https://api.spotify.com/v1/artists/2MDj296KJIfgWDNBtHzeFi',
      'id': '2MDj296KJIfgWDNBtHzeFi',
      'name': '11:11 Music Group',
      'type': 'artist',
      'uri': 'spotify:artist:2MDj296KJIfgWDNBtHzeFi'}]

Ahora, para empezar numeraremos el nombre de la canción y los artistas respectivamente.

In [54]:
for i,item in enumerate (top20['items']):
    print(i+1 , item['name'],'-',item['artists'][0]['name'])

1 Do I Wanna Know (8D Audio) - creamy
2 Everything Matters - AURORA
3 Blame Game - Mindchatter
4 Sex on Fire - Kings of Leon
5 Californication - Red Hot Chili Peppers
6 say goodbye - Sarcastic Sounds
7 De Pe a Pa - Granuja
8 Toxicity - System Of A Down
9 Noicanicula - La Etnnia
10 The Pretender - Foo Fighters
11 Ateo - Granuja
12 Roca N' Ron - Doble Porcion
13 It's Called: Freefall - Paris Paloma
14 Quién Pidió Pollo - Granuja
15 Faint - Linkin Park
16 Under Kings - Granuja
17 Días de Perros - Granuja
18 Falsedades - Tres Coronas
19 Something In The Way - Nirvana
20 El Emblema - Vic Deal


## Extracción de la información
Para empezar, nos interesa extraer información de cada una de las canciones del top 20 del usuario, a continuación crearemos un dataframe que guardará:
* Nombre de la canción
* Id de la canción 
* Caracteristicas de la canción

Un paso interesante es que, recién conocemos el ID de la canción vamos a buscar sus características usando el objeto ```sp```.

In [55]:
import pandas as pd # Importar pandas 

tracks = top20['items'] # Llamando el valor Items del diccionario Top 20
# Declarar listas vacías para guardar la información
track_ids = []
track_names = []
features = []
# Almacenar la información de las canciones de Top 20
for track in tracks:
    track_id = track['id'] # Guarda el ID de las canciones
    track_name = track['name'] # Guarda el nombre de las canciones
    audio_features = sp.audio_features(track_id) # Guarda las caracteristicas de las canciones partir de su ID
    
    track_ids.append(track_id)
    track_names.append(track_name)
    features.append(audio_features[0])

In [56]:
top20_df = pd.DataFrame(features,index = track_names)
top20_df.head(n = 2)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
Do I Wanna Know (8D Audio),0.679,0.242,7,-15.781,0,0.0538,0.622,0.000023,0.1040,0.315,92.044,audio_features,6dvGAhdmKhA4XTXkmo588g,spotify:track:6dvGAhdmKhA4XTXkmo588g,https://api.spotify.com/v1/tracks/6dvGAhdmKhA4...,https://api.spotify.com/v1/audio-analysis/6dvG...,203983,4
Everything Matters,0.453,0.325,7,-9.758,0,0.0484,0.657,0.000065,0.0697,0.324,160.380,audio_features,1rMlnD8S0hX7IHZXZDt3gH,spotify:track:1rMlnD8S0hX7IHZXZDt3gH,https://api.spotify.com/v1/tracks/1rMlnD8S0hX7...,https://api.spotify.com/v1/audio-analysis/1rMl...,213907,3


In [57]:
top20_df = top20_df[["id","acousticness","danceability","duration_ms","energy","instrumentalness","key","liveness","loudness","mode","speechiness","tempo","valence"]]
top20_df.head(n =2)

,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
Do I Wanna Know (8D Audio),6dvGAhdmKhA4XTXkmo588g,0.622,0.679,203983,0.242,0.000023,7,0.1040,-15.781,0,0.0538,92.044,0.315
Everything Matters,1rMlnD8S0hX7IHZXZDt3gH,0.657,0.453,213907,0.325,0.000065,7,0.0697,-9.758,0,0.0484,160.380,0.324


# Listado de canciones candidatas
Ahora, a partir del Top 20 canciones más escuchadas vamos a generar un *listado de canciones candidatas*, para esto haremos uso de la API para poder obtener artistas similares, los pasos a seguir serán:
1. Tomar los artistas sin repeticiones del Top 20.
2. Crear la lista de *Artistas relacionados* usando la API  
    a. ```sp.artist_related_artists```   
    b. ```sp.new_releases```   
3. Crear la lista de *Albums* a partir de los artistas del paso anterior
4. Seleccionar canciones por cada *Album* del paso anterior

In [58]:
# 1. Extraer los artistas correspondientes al top 20 (sin repeticiones)
ids_artists = []
print('Artistas en mi top20:')
print('=====================')
for item in top20['items']:
    artist_id = item['artists'][0]['id']
    artist_name = item['artists'][0]['name']
    print(f'{artist_id}: {artist_name}')
    ids_artists.append(artist_id)
    
# Eliminar repetidos
ids_artists = list(set(ids_artists))
print(f'Número de artistas (sin repeticiones): {len(ids_artists)}')

Artistas en mi top20:
2uSVRVjYZ0PlJPdFHnBETu: creamy
1WgXqy2Dd70QQOU7Ay074N: AURORA
1He0ZKninbT4FMEV9hUZKn: Mindchatter
2qk9voo8llSGYcZ6xrBzKx: Kings of Leon
0L8ExT028jH3ddEcZwqJJ5: Red Hot Chili Peppers
1bq8rqNnfrojn0OSAfeNXJ: Sarcastic Sounds
5KBPxIED8ejHkvhs4KCzyS: Granuja
5eAWCfyUhZtHHtBdNk56l1: System Of A Down
5g03v1dieHbVUP47XeXk7H: La Etnnia
7jy3rLJdDQY21OgRLCZ9sD: Foo Fighters
5KBPxIED8ejHkvhs4KCzyS: Granuja
4PLGokgPkAapRsKDwXq408: Doble Porcion
2EXpthNgSeTDeX8nGwxppp: Paris Paloma
5KBPxIED8ejHkvhs4KCzyS: Granuja
6XyY86QOPPrYVGvF9ch6wz: Linkin Park
5KBPxIED8ejHkvhs4KCzyS: Granuja
5KBPxIED8ejHkvhs4KCzyS: Granuja
5kROocD0ED0BYdVo7tJrSe: Tres Coronas
6olE6TJLqED3rqDCT0FyPh: Nirvana
0NcNqG01boinOKgb7ImmEJ: Vic Deal
Número de artistas (sin repeticiones): 16


In [59]:
# 2. a. Crear lista de artistas relacionados usando la API
print('Artistas similares:')
print('=====================')
ids_similar_artists = []
for artist_id in ids_artists:
    artists = sp.artist_related_artists(artist_id)['artists']
    for item in artists:
        artist_id = item['id']    # Extraer ID de artista relacionado
        artist_name = item['name'] # Extraer nombre de artista relacionado
        print(f'{artist_id}: {artist_name}')
        ids_similar_artists.append(artist_id) # Agregar a la lista el artista relacionado
        
# Agregar artistas del top 20 a la lista de artistas candidatos
ids_artists.extend(ids_similar_artists)
# Eliminar repetidos
ids_artists = list(set(ids_artists))
print(f'Número de artistas (sin repeticiones): {len(ids_artists)}')

Artistas similares:
3ygJTpJJIK7eEeC2EFRl9D: Alcolirykoz
0cRHRB8jrBv74aDfyXQPrd: Rap Bang Club
6nfNQvqechLUJaFHYAciep: Aerophon
6i28S1MHCSNJtufoq0DnPv: No Rules Clan
3fQP5a7SIC91kV4N8AOy53: Crudo Means Raw
6oqagavQDMpR3KKeX4eqjL: N. Hardem
6kfPkheKpxesJXLqA4h7jO: Tsh Sudaca
7zXRgSpjJkQmB9SnPLq1Qk: Crack Family
1mVVNTPZrLkxfnB7Wc1Xh1: Anyone/Cualkiera
78rkeQAvjl85yNyLAqEMVe: Mañas Ru-Fino
1xS8E28QGLkEoXlJdY94hq: Sr Pablo
10MeG72hSqEKGbUdBowf9k: Métricas Frías
5Ti7e3WlCPPlcsZfxqtzTo: Ruzto
2XNJ1dXD5AjaHZTSM7qPlH: Margarita Siempre Viva
0NcNqG01boinOKgb7ImmEJ: Vic Deal
30IozP2GkHKFl6U59F86lF: Frank Takuma
6ppLSXJZysRO3LBrlJIVuA: Soul AM
5ZdWGWlHFXgo51ouiol6hl: Ali Aka Mind
4mSxhskrhEcz1mZNVf1GWa: Kiño
02zSCw0p861wNWUjuC2jc3: Zof Ziro
05fG473iIaoy82BF1aGhL8: Slipknot
3RNrq3jvMZxD9ZyoOZbQOD: Korn
0BEI7i5sgUuivcfwXLzFmM: Serj Tankian
3TOqt5oJwL9BE2NG9MEwDa: Disturbed
2Pfv2w8a20xzC7Dr7QXRqM: Mudvayne
5vbtZfRJjn2QLhPhtYpiqi: Daron Malakian and Scars On Broadway
49qiE8dj4JuNdpYGRPdKbF: Stone Sou

31mtcsgS3CJW5dEPlG83E7: untrusted
42tsiNwD1xyyE4Hws2RvrC: Jasper
2MDj296KJIfgWDNBtHzeFi: 11:11 Music Group
1U0VLqR5nhfKiS3ngoqeZm: daddy's girl
6U65hm7m0hVAVGV0SXNFMw: ghostly echoes
5ac82nrmEXLxYO5Aa88aH7: Jess Taggart
6NYAOYclXxCr9rkMK8T8de: 8D sped up viral audios
4bGPjihFEUsxf9LPn1epOv: Martin Arteta
2v3eLcVAnCd6Yjeu0TzeGS: Hloshit
33FWu1JTA73HpPE2tBiwNX: Addict.
62AlnDIQccDoQ5FPZRrSL3: Solace
6YnaF6icmmm03VSrrM28ft: boy in skirt
59yO7tLFDTBR217x4u8iJn: pretence
1mVFtRG3DGigjQYeET4aJx: Yehu
1z0Ka2bQ4pcy89QzXw8dUa: sped up tiktok audios
2h36lqLlhIyJt4UpCJrLG5: meynus
1UkHuDUhU5Ik4cNH6F4XUq: Raspo
3O7oOXK7KPniAPEUorctsi: Sarah Meow
1N8Y4kaIF0ktxmylgsvUiY: Rewind
0K3Jtj8PRdPfgYLua8X1dv: Yusei
78SHxLdtysAXgywQ4vE0Oa: Bush
2ziB7fzrXBoh1HUPS6sVFn: Audioslave
5xUf6j4upBrXZPg6AI4MRK: Soundgarden
2UazAtjfzqBF0Nho2awK4z: Stone Temple Pilots
40Yq4vzPs9VNUrIBG5Jr2i: The Smashing Pumpkins
0iHb0mCbqZTYeb4y9Pirrd: Temple Of The Dog
64tNsm6TnZe2zpcMVMOoHL: Alice In Chains
4iudEcmuPlYNdbP3e1bdn1: S

In [60]:
# 2. b. Crear listas de artistas con nuevos lanzamientos usando la API
print('')
print('Artistas con nuevos lanzamientos:')
print('=====================')
new_releases = sp.new_releases(limit=20)['albums'] # Llamando los nuevos lanzamientos a partir del objeto
for item in new_releases['items']:
    artist_id = item['artists'][0]['id']   #[0] porque puede haber varios artistas, se tomará el primero
    artist_name = item['artists'][0]['name']
    album_name = item['name']   # Nombre del album, puramente informativo
    release_date = item['release_date'] # Fecha de lanzamiento, puramente informativo
    print(f'{artist_id}: {artist_name} - // {album_name}, {release_date}')
    ids_artists.append(artist_id)
    
# Eliminar repetidos
ids_artists = list(set(ids_artists))
print(f'Número de artistas (sin repeticiones): {len(ids_artists)}')


Artistas con nuevos lanzamientos:
2PfyKA4qhjkxUVkerTCxz0: Juan Pablo Vega - // Despídeme De Todxs, 2023-05-11
7gOdHgIoIKoe4i9Tta6qdD: Jonas Brothers - // The Album, 2023-05-12
6dexNK5MjEL8UvmA5MjSgg: Felipe Peláez - // Un Sueño Llamado RANCHERA, 2023-05-12
4tZwfgrHOc3mvqYlEYSvVi: Daft Punk - // Random Access Memories (10th Anniversary Edition), 2023-05-12
6eUKZXaKkcviH0Ku9w2n3V: Ed Sheeran - // - (Deluxe), 2023-05-05
0sXJRmgbjbq6Q5uu4W1wDM: Las Villa - // Eclipse, 2023-05-04
50sIhX3HytFEwQXZJLUZQE: Alejo - // EL FAVORITO DE LAS NENAS, 2023-05-04
1phfTBIocBW3UwqcYjaEN6: Mike Bahía - // Contigo, 2023-04-27
36KsCCwgI0Dep97yVJWmkK: Monsieur Periné - // Bolero Apocalíptico, 2023-04-27
22p8vOZwMABvl5qt2nZHWD: Vale - // A Contraluz, 2023-04-28
4vhNDa5ycK0ST968ek7kRr: Carlos Vives - // Escalona Nunca Se Había Grabado Así, 2023-04-18
68LgpWsaAwjflP3CLXC0LB: Susana Cala - // Sin Mensaje De Buenos Días, 2023-04-20
2o5jDhtHVPhrJdv3cEQ99Z: Tiësto - // DRIVE, 2023-04-21
2ye2Wgw4gimLv2eAKyk1NB: Meta

Vamos a crear la lista de albums, para esto vamos a ir a obtener el nombre de cada album de cada artista, y es esos respectivos albums buscar canciones similares a las del Top 20, eso sería lo ideal, pero por ahora, vamos a trabajar con un album.

In [66]:
# 3. Crear la lista de albums 
id_albums = []
nartists = len(ids_artists)
for i, id_artist in enumerate(ids_artists):
    print(f'Procesando artista {i+1} de {nartists}...')
    albums = sp.artist_albums(id_artist, limit=1) # Nos limita a un Album por artista
    for album in albums['items']:
        id_albums.append(album['id'])
print('¡Listo!')

Procesando artista 1 de 266...
Procesando artista 2 de 266...
Procesando artista 3 de 266...
Procesando artista 4 de 266...
Procesando artista 5 de 266...
Procesando artista 6 de 266...
Procesando artista 7 de 266...
Procesando artista 8 de 266...
Procesando artista 9 de 266...
Procesando artista 10 de 266...
Procesando artista 11 de 266...
Procesando artista 12 de 266...
Procesando artista 13 de 266...
Procesando artista 14 de 266...
Procesando artista 15 de 266...
Procesando artista 16 de 266...
Procesando artista 17 de 266...
Procesando artista 18 de 266...
Procesando artista 19 de 266...
Procesando artista 20 de 266...
Procesando artista 21 de 266...
Procesando artista 22 de 266...
Procesando artista 23 de 266...
Procesando artista 24 de 266...
Procesando artista 25 de 266...
Procesando artista 26 de 266...
Procesando artista 27 de 266...
Procesando artista 28 de 266...
Procesando artista 29 de 266...
Procesando artista 30 de 266...
Procesando artista 31 de 266...
Procesando artist

Procesando artista 253 de 266...
Procesando artista 254 de 266...
Procesando artista 255 de 266...
Procesando artista 256 de 266...
Procesando artista 257 de 266...
Procesando artista 258 de 266...
Procesando artista 259 de 266...
Procesando artista 260 de 266...
Procesando artista 261 de 266...
Procesando artista 262 de 266...
Procesando artista 263 de 266...
Procesando artista 264 de 266...
Procesando artista 265 de 266...
Procesando artista 266 de 266...
¡Listo!


> No recomendado para pruebas, solo para versión final     
Ahora, trabajaremos con más albums por artista, sin embargo, sin mostrar el proceso. **(Proceso demorado)**

# OMISIBLE SI SE ESTA PROBANDO
id_albums = []
nartists = len(ids_artists)
for i, id_artist in enumerate(ids_artists):
    albums = sp.artist_albums(id_artist, limit=3) # Ahora elegiremos 3 albums por artista
    for album in albums['items']:
        id_albums.append(album['id'])
print('¡Listo!')

In [67]:
id_albums[0:5] # Recordar que estamos trabajando con IDs

['6GJRkKGuVxmPJ3QlOFh7wK',
 '0ECbZXtdH1uFh7WYiyAAmm',
 '2sAePf08xIp4tnDlMUCV8B',
 '4fN6xfIrXPX8SMgcFU4A6h',
 '50CtNizqlxOXeHICmNwc45']

Vamos a extraer por cada album, 3 canciones, esto con el fin de qué el procesamiento no sea tan duro.

In [68]:
# 4. 
id_tracks = []
nalbums = len(id_albums)
for i, id_album in enumerate(id_albums):
    album_tracks = sp.album_tracks(id_album, limit=3)
    for track in album_tracks['items']:
        id_tracks.append(track['id'])
print(f'¡Listo! Número total de canciones pre-candidatos: {len(id_tracks)}')

¡Listo! Número total de canciones pre-candidatos: 765


Y por último, ya teniendo las canciones que tenemos como candidatas, vamos a construir un *DataFrame* similar a de Top 20 pero con las canciones candidatas, y además, haciendo uso de las caracteristica que nos brinda la API. 
> Este proceso puede tardar, varía de acuerdo a la velocidad del internet y a las caracteristica del dispositivo. 

In [69]:
track_names = []
features = []
ntracks = len(id_tracks)
for i, track_id in enumerate(id_tracks):
    #print(f'Procesando track {i+1} de {ntracks}...') # Si se quiere ver el proceso, descomentar linea
    track_name = sp.track(track_id)['name']
    audio_features = sp.audio_features(track_id)
    
    #No incluir pistas sin "features"
    if audio_features[0] != None:
        track_names.append(track_name)
        features.append(audio_features[0])
print('¡Listo!')

candidatos_df = pd.DataFrame(features,index = track_names)

¡Listo!


In [70]:
candidatos_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
Nowbody Knows,0.773,0.611,7,-7.140,0,0.3260,0.7880,0.000762,0.0956,0.910,82.087,audio_features,6yyVtOyIx6NHQ6bkMVoMFS,spotify:track:6yyVtOyIx6NHQ6bkMVoMFS,https://api.spotify.com/v1/tracks/6yyVtOyIx6NH...,https://api.spotify.com/v1/audio-analysis/6yyV...,127564,4
La Cosecha (Sascha Fitness),0.811,0.506,5,-7.930,1,0.4980,0.5720,0.000000,0.1090,0.673,82.107,audio_features,2FNxYvwej1pkeg3ZeAqDcf,spotify:track:2FNxYvwej1pkeg3ZeAqDcf,https://api.spotify.com/v1/tracks/2FNxYvwej1pk...,https://api.spotify.com/v1/audio-analysis/2FNx...,174335,4
Lilo & Stitch,0.755,0.492,2,-7.397,1,0.7780,0.7650,0.000000,0.0864,0.744,78.152,audio_features,2acLGzDcr3YdzBKUUUSKq5,spotify:track:2acLGzDcr3YdzBKUUUSKq5,https://api.spotify.com/v1/tracks/2acLGzDcr3Yd...,https://api.spotify.com/v1/audio-analysis/2acL...,157172,4
Am I Really Going To Die,0.504,0.965,1,-4.357,1,0.0669,0.0172,0.103000,0.1520,0.239,110.028,audio_features,528XjXC7Xz0qQtyyI8etu9,spotify:track:528XjXC7Xz0qQtyyI8etu9,https://api.spotify.com/v1/tracks/528XjXC7Xz0q...,https://api.spotify.com/v1/audio-analysis/528X...,287960,4
As I Try Not To Fall Apart,0.625,0.779,7,-6.001,0,0.0317,0.0833,0.006770,0.0862,0.888,116.988,audio_features,2rgWJThk3MFyb7WCcZCzpV,spotify:track:2rgWJThk3MFyb7WCcZCzpV,https://api.spotify.com/v1/tracks/2rgWJThk3MFy...,https://api.spotify.com/v1/audio-analysis/2rgW...,298680,4


In [71]:
# Reorganizar caracteristicas de las canciones para que queden igual al DataFrame del Top 20
candidatos_df = candidatos_df[["id","acousticness","danceability","duration_ms","energy","instrumentalness","key","liveness","loudness","mode","speechiness","tempo","valence"]]
candidatos_df.head(n =2)

,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
Nowbody Knows,6yyVtOyIx6NHQ6bkMVoMFS,0.788,0.773,127564,0.611,0.000762,7,0.0956,-7.14,0,0.326,82.087,0.910
La Cosecha (Sascha Fitness),2FNxYvwej1pkeg3ZeAqDcf,0.572,0.811,174335,0.506,0.000000,5,0.1090,-7.93,1,0.498,82.107,0.673


In [72]:
candidatos_df.shape

(764, 13)

In [73]:
top20_df.shape

(20, 13)

In [133]:
top20_df.head()

,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
Do I Wanna Know (8D Audio),6dvGAhdmKhA4XTXkmo588g,0.62200,0.679,203983,0.242,0.000023,7,0.1040,-15.781,0,0.0538,92.044,0.315
Everything Matters,1rMlnD8S0hX7IHZXZDt3gH,0.65700,0.453,213907,0.325,0.000065,7,0.0697,-9.758,0,0.0484,160.380,0.324
Blame Game,1WqgjwwlLRKE6qtcUY37S1,0.67300,0.668,201255,0.592,0.756000,9,0.0880,-12.645,1,0.1390,179.949,0.488
Sex on Fire,0ntQJM78wzOLVeCUAW7Y45,0.00172,0.542,203347,0.905,0.010400,9,0.1360,-5.653,1,0.0540,153.398,0.374
Californication,48UPSzbZjgc449aqz8bxox,0.00210,0.592,329733,0.767,0.001650,9,0.1270,-2.788,0,0.0270,96.483,0.328


In [135]:
candidatos_df.head()

,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
Nowbody Knows,6yyVtOyIx6NHQ6bkMVoMFS,0.7880,0.773,127564,0.611,0.000762,7,0.0956,-7.140,0,0.3260,82.087,0.910
La Cosecha (Sascha Fitness),2FNxYvwej1pkeg3ZeAqDcf,0.5720,0.811,174335,0.506,0.000000,5,0.1090,-7.930,1,0.4980,82.107,0.673
Lilo & Stitch,2acLGzDcr3YdzBKUUUSKq5,0.7650,0.755,157172,0.492,0.000000,2,0.0864,-7.397,1,0.7780,78.152,0.744
Am I Really Going To Die,528XjXC7Xz0qQtyyI8etu9,0.0172,0.504,287960,0.965,0.103000,1,0.1520,-4.357,1,0.0669,110.028,0.239
As I Try Not To Fall Apart,2rgWJThk3MFyb7WCcZCzpV,0.0833,0.625,298680,0.779,0.006770,7,0.0862,-6.001,0,0.0317,116.988,0.888


## Inicio versión 3

Para comenzar a elegir las canciones recomendadas, vamos a usar un *Sistema de recomendación basado en contenido*, lo cuál implica que compararemos las caracteristicas sonoras del Top 20 canciones del usuario con las **canciones candidatas** elegidas anteriormente.

In [136]:
# Tomar solo las caracteristicas de las canciones para poder así compararlas
top20_mtx = top20_df.iloc[:,1:].values
candidatos_mtx = candidatos_df.iloc[:,1:].values

Como estamos hablando de vectores de 13 dimensiones, donde cada una de sus componentes tiene su propia escala, lo primero que debemos hacer es estandarizarlas.


In [75]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
t20_scaled = scaler.fit_transform(top20_mtx)
can_scaled = scaler.fit_transform(candidatos_mtx)

Ahora, tenemos vectores estandarizados, pero para poder comparlos entre ellos necesitamos normalizarlos, lo que haremos a continuación:


In [78]:
import numpy as np

# Raíz de la suma
t20_norm = np.sqrt((t20_scaled*t20_scaled).sum(axis=1))
can_norm = np.sqrt((can_scaled*can_scaled).sum(axis=1))
# División de la norma
nt20 = t20_scaled.shape[0]
ncan = can_scaled.shape[0]
t20 = t20_scaled/t20_norm.reshape(nt20,1)
can = can_scaled/can_norm.reshape(ncan,1)

El método sencillo pero efectivo usado frecuentemente en el Machine Learning es el de similitud de Coseno, donde al comparar dos vectores si se asemeja a 1 el coseno quiere decir que son similares, si se asemeja a 0 no tienen similaridad y si se asemejan a -1 son opuestos, usaremos este metodos para comparar ambos vectores.

In [87]:
from sklearn.metrics.pairwise import linear_kernel

cos_sim = linear_kernel(t20,can)
cos_sim.shape

(20, 764)

In [137]:
cos_sim

array([[ 0.05576776, -0.14255593, -0.17809125, ...,  0.14050658,
         0.12495708,  0.4154892 ],
       [-0.27272256, -0.54067646, -0.42136578, ..., -0.41964054,
        -0.46690317, -0.19612895],
       [-0.20503386, -0.17261836, -0.13442285, ..., -0.19536016,
        -0.17503299, -0.32228304],
       ...,
       [-0.06037676,  0.04879765,  0.06891333, ..., -0.0471908 ,
         0.05199028, -0.13966538],
       [-0.48749083, -0.34094901, -0.30458395, ..., -0.55254409,
        -0.02122197, -0.27298757],
       [ 0.75877132,  0.42644591,  0.39291437, ...,  0.76322559,
         0.24972556,  0.46697344]])

Tenemos una matriz de 20 filas, es decir, el top 20 del usuario, y las comparamos con cada una de las canciones candidatas.  Ahora lo que tenemos que hacer es buscar las canciones que tengan más similitud con las canciones del top 20, para eso usaremos la siguiente función.

In [89]:
# Obtener candidatos para una canción
# pos -> posicion de la canción del top 20
# cos_sim -> la matriz de cosenos 
# n -> candidatos número de candidatos por canción del top 20
# umbral -> Nivel de similitud a la referencial 
def obtener_candidatos(pos, cos_sim, ncandidatos, umbral = 0.8):
    # Obtener todas las pistas candidatas por encima de umbral
    idx = np.where(cos_sim[pos,:]>=umbral)[0] # ejm. idx: [27, 82, 135]
    
    # Y organizarlas de forma descendente (por similitudes de mayor a menor)
    idx = idx[np.argsort(cos_sim[pos,idx])[::-1]] # [::-1] porque por defecto argsort organiza de manera ascendente

    # Si hay más de "ncandidatos", retornar máximo "ncandidatos"
    if len(idx) >= ncandidatos:
        candidatos = idx[0:ncandidatos]
    else:
        candidatos = idx
  
    return candidatos

In [90]:
# Prueba
for i in range(5):
    cands = obtener_candidatos(i, cos_sim, 5)
    print(f'{i}==> pistas filtradas: {cands}, similitudes: {cos_sim[i,cands]}')

0==> pistas filtradas: [529 634 339 530  57], similitudes: [0.92590158 0.86290426 0.8525872  0.83470603 0.83116108]
1==> pistas filtradas: [659], similitudes: [0.83776351]
2==> pistas filtradas: [], similitudes: []
3==> pistas filtradas: [194 189 284 180 286], similitudes: [0.90355652 0.90221597 0.9021726  0.88812643 0.8752983 ]
4==> pistas filtradas: [289 332 159 531], similitudes: [0.90958809 0.88337997 0.82023109 0.81579706]


A continuación, guardaremos en una lista las pistas similares a las del top 20, imprimiendo además cuales canciones de los *artistas relacionados* y de los *nuevos lanzamientos* fueron seleccionadas por cada canción del Top 20.

In [96]:
ids_t20 = []
ids_playlist = []

for i in range(top20_df.shape[0]):
    print(f'{i+1}. {top20_df.index[i]}')   # Nombre de la pista en el top-20
    ids_t20.append(top20_df['id'][i])
    
    # Obtener listado de candidatos para esta pista
    cands = obtener_candidatos(i, cos_sim, 5, umbral=0.8)
    
    # Si hay pistas relacionadas obtener los ids correspondientes e imprimir en pantalla
    if len(cands)==0:
        print('     ***No se encontraron pistas relacionadas***')
    else:
        # Obtener los ids correspondientes e imprimir en pantalla
        for j in cands:
            id_cand = candidatos_df['id'][j]
            ids_playlist.append(id_cand)
            
            # E imprimir en pantalla el candidato
            print(f'   * {candidatos_df.index[j]}')

1. Do I Wanna Know (8D Audio)
   * Confined
   * supermodel
   * Exhale Inhale
   * Monalogue
   * I Kissed Someone (It Wasn't You)
2. Everything Matters
   * Carrion Comfort - Slowed
3. Blame Game
     ***No se encontraron pistas relacionadas***
4. Sex on Fire
   * Attack
   * Under You
   * You Don't Know
   * Machine (Ft. Gary Stringer)
   * Go to Hell
5. Californication
   * Screaming Suicide
   * Heart Attack
   * State Of The Art
   * The One You Know
6. say goodbye
   * Una Preguntita (Intro)
7. De Pe a Pa
   * Hasta el Último Round
   * Noche de Bohemia
   * La Vida Es Corta
8. Toxicity
   * Under You
   * Machine (Ft. Gary Stringer)
   * Take Me Out
   * Slow Me
   * Rescued
9. Noicanicula
   * Walk
   * Lilo & Stitch
   * La Cosecha (Sascha Fitness)
10. The Pretender
   * You Don't Know
   * Go to Hell
   * Under You
   * Rescued
   * Attack
11. Ateo
   * Una Preguntita (Intro)
12. Roca N' Ron
   * Derrapando
   * Sguardi
13. It's Called: Freefall
   * Más o Menos Antes
   * 

In [98]:
# Eliminar canciones recomendadas que ya hacen parte del Top 20
ids_playlist_dep = [x for x in ids_playlist if x not in ids_t20]
ids_playlist_dep = list(set(ids_playlist_dep))

In [131]:
print('Canciones Recomendadas:')
print('=====================')
for item in ids_playlist_dep:
    track = sp.track(item)
    song_name = track['name']
    artist_name = track['artists'][0]['name']
    print(song_name, end=' - ')
    print(artist_name)
    
    
    #print(sp.track(item)['name'])
    #print(sp.track(item)['artists'][0]['name'])
    
# Buscar la canción por su ID
#track = sp.track(song_id)

# Obtener el nombre del artista
#artist_name = track['artists'][0]['name']
    


Canciones Recomendadas:
Start The Healing - Requiem Mass - Korn
I Still Don't Know Who I Am. - Barnes Blvd.
Más o Menos Antes - Silvana Estrada
Not Again - Live - Staind
La Cosecha (Sascha Fitness) - Soul AM
Monalogue - Timmies
Tonos Grises - Métricas Frías
Aprovéchame - Yoky Barrios
Noche de Bohemia - Laberinto ELC
Attack - System Of A Down
State Of The Art - Incubus
Walk - potsu
Derrapando - Cráneo
Picturesque - Attom
HYACINTHUS - Aidoneus
Gasoline - Seether
Carrion Comfort - Slowed - aeseaes
Z28 - Static-X
Hue - Attom
La Vida Es Corta - Clan Hueso Duro
Pido permiso - Nach Scratch
You Don't Know - Puddle Of Mudd
Hasta el Último Round - Superanfor
Una Preguntita (Intro) - Los Elefantes
Rescued - Foo Fighters
So Called Life - Three Days Grace
Under You - Foo Fighters
Slow Me - Bush
Wait for Me - AlicebanD
I Kissed Someone (It Wasn't You) - dodie
Machine (Ft. Gary Stringer) - Skindred
Sguardi - Fedi
Terrible Thing - The Killers
you look like summer - Askjell
Caciques del Altiplano - Ali